<a href="https://colab.research.google.com/github/Hameon4/Kaggle-2022-GAN/blob/main/Kaggle_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PART 1: BUILD MODEL USING THE VALIDATION AND SOLUTION DATASET**

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/content/Train.csv')

In [3]:
X = df.drop(columns = ['Label']).copy()
y = df['Label']

In [4]:
# Build the model
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(n_estimators=500, criterion='entropy')
model.fit(X, y)

ExtraTreesClassifier(criterion='entropy', n_estimators=500)

In [5]:
data_valid = pd.read_csv('/content/Validation.csv')
data_solution = pd.read_csv('/content/Solution.csv')

X_valid = data_valid.drop(columns = ['ID']).copy()
y_valid = data_solution.drop(columns = ['ID']).copy()

In [6]:
from sklearn.metrics import mean_squared_error

p_test = model.predict(X_valid)
rmse = mean_squared_error(y_valid, p_test, squared=False)
print(f'RMSE: {round(rmse * 100, 3)}%')


RMSE: 11.698%


# **PART 2: UTILIZE GAN TO GENERATE SYNTHETIC DATA**

In [7]:
import pandas as pd
data = pd.read_csv('/content/Train.csv')

In [8]:
x = data['Label']
z = 0
o = 0

for i in x:
  if i == 0:
    z += 1
  elif i == 1:
    o += 1
print(f'Zeros: {z} Ones: {o}')

Zeros: 3000 Ones: 1465


In [ ]:
!pip install sdv

In [ ]:
from sdv.tabular import CTGAN
model = CTGAN(epochs=10)
model.fit(data)

In [15]:
from sdv.sampling import Condition
synthetic_data = Condition({
    'Label':1,
}, num_rows= z - o)
synthetic_data = model.sample_conditions(conditions=[synthetic_data])

Sampling conditions: 100%|██████████| 1535/1535 [00:03<00:00, 442.17it/s]


In [18]:
# merge fake with original
malware_data = data.append(synthetic_data)

In [19]:
shuffled_malware_data = malware_data.sample(frac=1)

In [20]:
from sklearn.model_selection import train_test_split

X = shuffled_malware_data.drop(columns = ['Label']).copy()
y = shuffled_malware_data['Label']

In [21]:
# Build the model
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(n_estimators=500, criterion='entropy')
model.fit(X, y)

ExtraTreesClassifier(criterion='entropy', n_estimators=500)

In [22]:
data_valid = pd.read_csv('/content/Validation.csv')
data_solution = pd.read_csv('/content/Solution.csv')

X_valid = data_valid.drop(columns = ['ID']).copy()
y_valid = data_solution.drop(columns = ['ID']).copy()

In [25]:
from sklearn.metrics import mean_squared_error

p_test = model.predict(X_valid)
rmse_gan = mean_squared_error(y_valid, p_test, squared=False)
print(f'RMSE: {round(rmse_gan * 100, 3)}%')

RMSE: 16.859%
